In [4]:
import praw
import pandas as pd
import sys
sys.path.append("C:\\Users\\Tiziano Pacifico\\Desktop\\CodeTesi\\FASE 1 DATASET")
import credential

# Note

Il dataset id_utc_.json viene qui utilizato come base per l'estrazione delle informazioni direttamente con l'api di reddit e il wrapper di python praw. In questo modo non utilizziamo praw come base per l'estrazione di tutte le submission in quanto potremmo incorrere in alcune limitazioni temporali insite nelle api di reddit ma utilizziamo l'id delle submissions ottenute tramite l'api di pushshift. 
La struttura del dataframe contenente le informazioni di pertinenza è la seguente:
ID - AUTHOR - CREATED_UTC - EDITED - NUM_COMMENTS - SELF_TEXT - TITLE - UTC_STR
Il significato di ognuno dei campi è intuibile dalla label del campo stesso.
Per gli autori è stato optato di ottenere il nickname piuttosto che l'id del loro nome. Questo perchè abbiamo notato il sollevars di molte eccezioni relativamente al tentativo di recuperare l'id del redditor rispetto al nickname. Le successive analisi non avranno bisogno della costruzione di un dataframe di redditor, per il quale l'id sarebbe stata più comoda del nickname, ma nel caso si volesse procedere in quella direzione le informazioni salvate saranno sufficienti per l'estrazione dei dati relativi.
Tecnicamente abbiamo iterato direttamente sulla lista di id delle submissions utitlizzanto una chiamata all'api di reddit per ottenere la submission relativa. 
Su ogni submission abbiamo utilizzato un dizionario per creare una riga del dataframe e abbiamo inserito la riga del dataframe direttamente all'interno del ciclo. Il sistema si è rivelato piuttosto lento per le ragioni già esposte in PushshiftPostDowload.
Il dataframe così ottenuto è stato df_submissions.json, contentente 73943 entries di cui 0 null per ognuno dei campi di interesse.

In [47]:
df_cols = ['id','author', 'created_utc','edited','num_comments','self_text','title','utc_datetime_str','blob']

In [6]:
reddit = praw.Reddit(client_id = credential.CLIENT_ID,
                     client_secret = credential.CLIENT_SECRET,
                     user_agent = 'fetching submission: u/Ardenzio')

In [9]:
df = pd.read_json('C:\\Users\\Tiziano Pacifico\\Desktop\\TESI\\RedditDS\\id_utc.json', lines = True)

In [13]:
id_list = list(df['id'])

In [79]:
df_submissions = pd.DataFrame(columns=df_cols[0:7])

In [80]:
row = {}
for submission_id in id_list:
    submission = reddit.submission(id=submission_id)
    if submission:
        row[df_cols[0]] = submission.id
        if submission.author:
            row[df_cols[1]] = submission.author.name
        row[df_cols[2]] = int(submission.created_utc)
        row[df_cols[3]] = submission.edited
        row[df_cols[4]] = submission.num_comments
        row[df_cols[5]] = submission.selftext
        row[df_cols[6]] = submission.title
        df_submissions.loc[len(df_submissions)] = row
    else:
        print('qui')
        s = pd.Series([None,None,None,None,None,None,None,None],index=df_cols[0:7])
        df_submissions.loc[len(df)] = s

In [83]:
df_submissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73943 entries, 0 to 73942
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              73943 non-null  object
 1   author          73943 non-null  object
 2   author_premium  73943 non-null  int64 
 3   created_utc     73943 non-null  object
 4   edited          73943 non-null  int64 
 5   num_comments    73943 non-null  object
 6   self_text       73943 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.5+ MB


In [84]:
df_submissions.head()

,id,author,author_premium,created_utc,edited,num_comments,self_text
0,134stgc,tattedboi1_,1682960456,False,3,I don’t think Michael McIntyre has anything to...,ChatGPT does not do stand up comedy
1,134sq2h,cstein123,1682960372,False,1,A little over a week ago I released a tool cal...,Dreamcatcher - ChatGPT Retrieval on Steroids
2,134slvb,Technology342,1682960259,False,2,"[How to Jailbreak ChatGPT, List of Prompts](ht...",Thoughts on this?
3,134s4jz,WW_III_ANGRY,1682959802,False,1,**Me:** What insights and discoveries have yo...,Emergent abilities of chat GPT. We don't reall...
4,134rp29,NiceGuy-n2,1682959399,False,1,,This chatgpt app gives me a 20 free chat credits


In [85]:
df_submissions_change = df_submissions.rename(columns = {'author_premium':'created_utc', 
                                                        'created_utc': 'edited',
                                                        'edited': 'num_comments',
                                                        'num_comments': 'self_text',
                                                        'self_text': 'title'})

In [87]:
df_submissions_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73943 entries, 0 to 73942
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            73943 non-null  object
 1   author        73943 non-null  object
 2   created_utc   73943 non-null  int64 
 3   edited        73943 non-null  object
 4   num_comments  73943 non-null  int64 
 5   self_text     73943 non-null  object
 6   title         73943 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.5+ MB


In [89]:
df_submissions = df_submissions_change.astype({"edited": bool})

In [91]:
df_submissions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73943 entries, 0 to 73942
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            73943 non-null  object
 1   author        73943 non-null  object
 2   created_utc   73943 non-null  int64 
 3   edited        73943 non-null  bool  
 4   num_comments  73943 non-null  int64 
 5   self_text     73943 non-null  object
 6   title         73943 non-null  object
dtypes: bool(1), int64(2), object(4)
memory usage: 4.0+ MB


In [92]:
df_submissions.to_json('C:\\Users\\Tiziano Pacifico\\Desktop\\TESI\\RedditDS\\df_submissions.json', orient='records', lines=True)